In [2]:
import numpy as np

# 初期配置をリストで表記
initial = np.array(
    [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9],
    ]
)


# 数独を成形して表示する
def print_sudoku(sudoku):
    for i in range(len(sudoku)):
        line = ""
        if i == 3 or i == 6:
            print("---------------------")
        for j in range(len(sudoku[i])):
            if j == 3 or j == 6:
                line += "| "
            line += str(sudoku[i][j]) + " "
        print(line)


print_sudoku(initial)

5 3 0 | 0 7 0 | 0 0 0 
6 0 0 | 1 9 5 | 0 0 0 
0 9 8 | 0 0 0 | 0 6 0 
---------------------
8 0 0 | 0 6 0 | 0 0 3 
4 0 0 | 8 0 3 | 0 0 1 
7 0 0 | 0 2 0 | 0 0 6 
---------------------
0 6 0 | 0 0 0 | 2 8 0 
0 0 0 | 4 1 9 | 0 0 5 
0 0 0 | 0 8 0 | 0 7 9 


## バイナリ変数の定義
$9 \times 9 \times 9 = 729$ 個の変数を用意する。

In [3]:
from amplify import VariableGenerator

gen = VariableGenerator()
q = gen.array("Binary", shape=(9, 9, 9))

In [4]:
# 行番号 $1$、列番号 $2$、レイヤ $3$ の変数
print(q[1, 2, 3])

# 行番号 $0$、列番号 $0$ の9変数
print(q[0, 0])

# 行番号 $2$、数値レイヤ $5$ の9変数
print(q[2, :, 5])

q_{1,2,3}
[q_{0,0,0}, q_{0,0,1}, q_{0,0,2}, q_{0,0,3}, q_{0,0,4}, q_{0,0,5}, 
 q_{0,0,6}, q_{0,0,7}, q_{0,0,8}]
[q_{2,0,5}, q_{2,1,5}, q_{2,2,5}, q_{2,3,5}, q_{2,4,5}, q_{2,5,5}, 
 q_{2,6,5}, q_{2,7,5}, q_{2,8,5}]


In [5]:
for i, j in zip(*np.where(initial != 0)):
    # 初期配置の i 行 j 列には数字が既に入っている
    k = initial[i, j] - 1

    q[i, j, :] = 0  # i 行 j 列のマスには数字 k+1 以外は入らない
    q[i, :, k] = 0  # 同じ行には数字 k+1 は入らない
    q[:, j, k] = 0  # 同じ列には数字 k+1 は入らない
    for m in range(9):
        # 同じブロックには数字 k+1 は入らない
        q[(3 * (i // 3) + m // 3), (3 * (j // 3) + m % 3), k] = 0

    q[i, j, k] = 1  # i 行 j 列のマスには数字 k+1 が入る

In [6]:
q[0,0]

PolyArray([0, 0, 0, 0, 1, 0, 0, 0, 0])

## 制約条件の設定
- 一つのマスには一つの数字しか入らない
$$
 \sum_{k=0}^8 q_{i,j,k} = 1
$$

In [7]:
from amplify import one_hot

# 一つのマスには一つの数字しか入らない制約条件
layer_constraints = one_hot(q, axis=2)

- 同じ行に同じ数字が入らない
$$
 \sum_{j=0}^8 q_{i,j,k} = 1
$$
- 同じ列に同じ数字が入らない
$$
 \sum_{i=0}^8 q_{i,j,k} = 1
$$

In [8]:
# 各行には同じ数字が入らない制約条件
row_constraints = one_hot(q, axis=1)

# 各列には同じ数字が入らない制約条件
col_constraints = one_hot(q, axis=0)

- $3 \times 3$ の各ブロックには同じ数字が入らない
$$
 \sum_{i, j \in \text{ブロック}} q_{i,j,k} = 1
$$

In [9]:
from amplify import sum as amplify_sum

# (c): 3x3ブロック内には同じ数字が入らない制約条件
block_constraints = amplify_sum(
    one_hot(amplify_sum([q[i + m // 3, j + m % 3, k] for m in range(9)]))
    for i in range(0, 9, 3)
    for j in range(0, 9, 3)
    for k in range(9)
)

## エネルギー関数

In [10]:
constraints = layer_constraints + row_constraints + col_constraints + block_constraints

## イジングマシンの実行

In [11]:
from amplify import solve, AmplifyAEClient
from datetime import timedelta

client = AmplifyAEClient()
client.parameters.time_limit_ms = timedelta(milliseconds=1000)  # タイムアウト 1000 ms
client.token = "AE/gd0y09RhHp0H0EcefLvtxYhbwl7Nk0US"

# 求解の実行
result = solve(constraints, client)

if len(result) == 0:
    raise RuntimeError("Some of the constraints are not satisfied.")

## 実行結果

In [ ]:
q_values = q.evaluate(result.best.values).astype(int)

# print(q_values[0, 0, 4])  # 左上のマスが 5 であるか
# print(q_values[4, 4, 0])  # 5 行 5 列のマス (盤面のちょうど真ん中のマス) が 1 であるか

In [13]:
answer = q_values @ np.arange(1, 10)
print_sudoku(answer)

5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
---------------------
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
---------------------
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 
